![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [91]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Import any additional modules and start coding below

In [92]:
#lets open the data and do some exploration
df = pd.read_csv('rental_info.csv')

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rental_date       15861 non-null  object 
 1   return_date       15861 non-null  object 
 2   amount            15861 non-null  float64
 3   release_year      15861 non-null  float64
 4   rental_rate       15861 non-null  float64
 5   length            15861 non-null  float64
 6   replacement_cost  15861 non-null  float64
 7   special_features  15861 non-null  object 
 8   NC-17             15861 non-null  int64  
 9   PG                15861 non-null  int64  
 10  PG-13             15861 non-null  int64  
 11  R                 15861 non-null  int64  
 12  amount_2          15861 non-null  float64
 13  length_2          15861 non-null  float64
 14  rental_rate_2     15861 non-null  float64
dtypes: float64(8), int64(4), object(3)
memory usage: 1.8+ MB
None


In [93]:
#lets create a feature called rental_length_days with return date and rental date, append it ot pandas

#set both cols to datetime so pandas understands its a date object

df['rental_date'] = pd.to_datetime(df['rental_date'])
df['return_date'] = pd.to_datetime(df['return_date'])

#now we can do some math on it

df['rental_length_days'] = (df['return_date'] - df['rental_date']).dt.days

print(df['rental_length_days'].head(3))

0    3
1    2
2    7
Name: rental_length_days, dtype: int64


In [94]:
df['deleted_scenes'] = np.where(df['special_features'].str.contains("Deleted Scenes"),1,0)

df['behind_the_scenes'] = np.where(df['special_features'].str.contains("Behind the Scenes"),1,0)

In [95]:
X = df.drop(columns=['rental_date', 'return_date', 'special_features', 'rental_length_days'])
y = df['rental_length_days']

In [96]:
#train test
X_train, X_Test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 9)

In [97]:
#lets use Lasso for the love of the game

from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.1, random_state=9)
lasso.fit(X_train, y_train)

coefs = lasso.coef_
mask = coefs != 0

X_train = X_train.iloc[:, mask]
X_test = X_Test.iloc[:, mask]

In [98]:
#now lets have a good list of models to try and test 

import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
"""
#model params list 
model_params = {
    'Linear Regression': (
        LinearRegression(),
        {'fit_intercept': [True, False]}
    ),
    
    'Random Forest': (
        RandomForestRegressor(random_state=9),
        {
            # range(start, stop) creates a list of integers
            'n_estimators': list(range(50, 200, 10)),  # [50, 60, ..., 190]
            'max_depth': [None, 10, 20, 30, 40],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
    ),
    
    'Decision Tree': (
        DecisionTreeRegressor(random_state=9),
        {
            'max_depth': [None, 5, 10, 20, 30],
            'min_samples_leaf': list(range(1, 10)),
            'max_features': ['sqrt', 'log2', None] 
        }
    ),
    
    'AdaBoost': (
        AdaBoostRegressor(random_state=9),
        {
            'n_estimators': list(range(50, 200, 10)),
            # np.linspace creates evenly spaced numbers
            'learning_rate': np.linspace(0.01, 1.0, 20), 
            'loss': ['linear', 'square', 'exponential']
        }
    )
}

best_models = {}

for name, (model, params) in model_params.items():
    print(f"Running search for {name}...")
    
    rs = RandomizedSearchCV(
        estimator=model,
        param_distributions=params,
        n_iter=20,             
        cv=5,                  
        scoring='neg_mean_squared_error',
        n_jobs=-1,             
        random_state=9
    )

    rs.fit(X_train, y_train)
    
    best_models[name] = rs.best_estimator_
    
    print(f"  Best Params: {rs.best_params_}")
    print(f"  Best Negative MSE: {rs.best_score_:.4f}\n")
"""


'\n#model params list \nmodel_params = {\n    \'Linear Regression\': (\n        LinearRegression(),\n        {\'fit_intercept\': [True, False]}\n    ),\n    \n    \'Random Forest\': (\n        RandomForestRegressor(random_state=9),\n        {\n            # range(start, stop) creates a list of integers\n            \'n_estimators\': list(range(50, 200, 10)),  # [50, 60, ..., 190]\n            \'max_depth\': [None, 10, 20, 30, 40],\n            \'min_samples_split\': [2, 5, 10],\n            \'min_samples_leaf\': [1, 2, 4]\n        }\n    ),\n    \n    \'Decision Tree\': (\n        DecisionTreeRegressor(random_state=9),\n        {\n            \'max_depth\': [None, 5, 10, 20, 30],\n            \'min_samples_leaf\': list(range(1, 10)),\n            \'max_features\': [\'sqrt\', \'log2\', None] \n        }\n    ),\n    \n    \'AdaBoost\': (\n        AdaBoostRegressor(random_state=9),\n        {\n            \'n_estimators\': list(range(50, 200, 10)),\n            # np.linspace creates even

In [99]:
#got best model params

#RandomForest
#{'n_estimators': 140, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 30}

best_model = RandomForestRegressor(
    n_estimators=140,
    min_samples_split=2,
    min_samples_leaf=1,
    max_depth=30
).fit(X_train, y_train)


y_pred = best_model.predict(X_test)

In [100]:
#now we get MSE

best_mse = mean_squared_error(y_test, y_pred)

print(best_mse)

2.1123024962147596
